# Define Imports

In [1]:
import pandas as pd
from google_play_scraper import Sort, reviews_all, reviews
from datetime import datetime
import time

## Define App Ids

In [2]:
APP_IDS = {
    'CBE': 'com.combanketh.mobilebanking',
    'BOA': 'com.boa.boaMobileBanking',
    'DASHEN': 'com.dashen.dashensuperapp'         
}

### Target Reviews Per App

In [3]:
TARGET_REVIEWS_PER_APP = 400

# Declaring Classes

## Playstore Scraper Class

In [4]:
class PlayStoreScraper:
    """
    A class to scrape app reviews from the Google Play Store.
    """
    def __init__(self, app_ids_dict):
        """
        Initializes the scraper with a dictionary of app names and their IDs.
        Args:
            app_ids_dict (dict): A dictionary where keys are bank names
                                 and values are their Google Play app IDs.
        """
        self.app_ids = app_ids_dict
        self.reviews_data = []

    def fetch_reviews_for_app(self, app_id, app_name, target_count=400, lang='en', country='us'):
        """
        Fetches a target number of reviews for a single app.
        Args:
            app_id (str): The Google Play ID of the app.
            app_name (str): The name of the bank/app.
            target_count (int): The desired number of reviews.
            lang (str): Language code for reviews.
            country (str): Country code for the Play Store.
        Returns:
            list: A list of review dictionaries.
        """
        print(f"Fetching reviews for {app_name} ({app_id})...")
        try:
            # Using 'reviews' which allows for 'count' parameter.
            # 'reviews_all' fetches all, which might be too many initially.
            
            app_reviews = []
            result, continuation_token = reviews(
                app_id,
                lang=lang,
                country=country,
                sort=Sort.NEWEST, # Or Sort.MOST_RELEVANT
                count=target_count + 200, # Fetch a bit more to account for filtering, duplicates if any from source
                filter_score_with=None # All scores
            )
            
            if not result:
                print(f"No reviews found for {app_name} with current settings.")
                return []

            for review in result:
                self.reviews_data.append({
                    'review_id': review.get('reviewId'),
                    'user_name': review.get('userName'),
                    'review': review.get('content'),
                    'rating': review.get('score'),
                    'date': review.get('at'), # This is a datetime object
                    'bank': app_name,
                    'source': 'Google Play Store' 
                })
                if len(self.reviews_data) % 100 == 0:
                    print(f"Collected {len(self.reviews_data)} reviews for {app_name} so far...")

            print(f"Successfully fetched {len(result)} reviews for {app_name}.")
           
        except Exception as e:
            print(f"Error fetching reviews for {app_id}: {e}")
        return result # Return the directly fetched reviews for this app

    def fetch_all_reviews(self, target_per_app):
        """
        Fetches reviews for all apps defined in app_ids.
        Args:
            target_per_app (int): Target number of reviews per app.
        """
        all_scraped_reviews = []
        for bank_name, app_id in self.app_ids.items():
            print(f"\nStarting scraping for: {bank_name}")

            fetched_app_reviews, _ = reviews(
                app_id,
                lang='en',
                country='et',     # Focusing on Ethiopian context
                sort=Sort.NEWEST, # Get the most recent feedback
                count=target_per_app + 50, # Fetch a bit more to be safe for the 400 target
                filter_score_with=None # Get all ratings
            )
            
            if fetched_app_reviews:
                for review_obj in fetched_app_reviews[:target_per_app]: # Take top N to ensure we don't vastly overshoot
                    all_scraped_reviews.append({
                        'review': review_obj.get('content'),
                        'rating': review_obj.get('score'),
                        'date': review_obj.get('at'), # This is a datetime object
                        'bank': bank_name,
                        'source': 'Google Play Store' # As per dataset overview 
                    })
                print(f"Collected {len(all_scraped_reviews)} reviews so far (target for {bank_name}: {target_per_app}).")
            else:
                print(f"Could not fetch reviews for {bank_name} ({app_id}).")
            
            time.sleep(2) # Basic courtesy delay

        self.reviews_df = pd.DataFrame(all_scraped_reviews)
        print(f"\nTotal reviews scraped: {len(self.reviews_df)}")
        return self.reviews_df

## Review Preprocessor Class

In [5]:
class ReviewPreprocessor:
    """
    A class to preprocess the scraped app reviews.
    """
    def __init__(self, df):
        """
        Initializes the preprocessor with a DataFrame of reviews.
        Args:
            df (pd.DataFrame): DataFrame containing the raw review data.
        """
        self.df = df.copy() # Work on a copy

    def normalize_dates(self, date_column='date', format='%Y-%m-%d'): # 
        """
        Normalizes the date column to YYYY-MM-DD format.
        Args:
            date_column (str): Name of the column containing date information.
            format (str): The desired output format for the date.
        """
        print("Normalizing dates...")
        if date_column in self.df.columns:
            self.df[date_column] = pd.to_datetime(self.df[date_column]).dt.strftime(format)
            print(f"'{date_column}' normalized.")
        else:
            print(f"Warning: Date column '{date_column}' not found.")
        return self

    def handle_missing_data(self): # 
        """
        Handles missing data in the DataFrame.
        For reviews, if 'review' text or 'rating' is missing, it's often best to drop.
        """
        print("Handling missing data...")
        initial_rows = len(self.df)
        # Drop rows where essential information like 'review' or 'rating' is missing
        self.df.dropna(subset=['review', 'rating'], inplace=True)
        rows_dropped = initial_rows - len(self.df)
        print(f"Dropped {rows_dropped} rows due to missing 'review' or 'rating'.")
        return self

    def remove_duplicates(self, subset_cols=None): # 
        """
        Removes duplicate reviews.
        Args:
            subset_cols (list, optional): List of column names to consider for identifying duplicates.
                                          Defaults to ['review', 'bank', 'date']
        """
        print("Removing duplicate reviews...")
        if subset_cols is None:
            subset_cols = ['review', 'rating', 'date', 'bank']
        
        initial_rows = len(self.df)
        self.df.drop_duplicates(subset=subset_cols, keep='first', inplace=True)
        rows_dropped = initial_rows - len(self.df)
        print(f"Dropped {rows_dropped} duplicate rows.")
        return self
        
    def ensure_columns(self):
        """Ensures the DataFrame has the required columns: review, rating, date, bank, source. """
        print("Ensuring required columns...")
        required_cols = ['review', 'rating', 'date', 'bank', 'source']
        # Select and reorder
        current_cols = [col for col in required_cols if col in self.df.columns]
        self.df = self.df[current_cols]
        
        # Check if all required columns are present after selection
        missing_cols = [col for col in required_cols if col not in self.df.columns]
        if missing_cols:
            print(f"Warning: The following required columns are still missing: {missing_cols}")
        else:
            print("All required columns are present.")
        return self


    def preprocess(self):
        """
        Runs the full preprocessing pipeline.
        """
        print("\n--- Starting Preprocessing ---")
        self.handle_missing_data()
        self.remove_duplicates() # Consider which columns define a unique review
        self.normalize_dates()
        self.ensure_columns() # Ensure final columns as per 
        print("--- Preprocessing Complete ---")
        return self.df

# Scrape the data

In [6]:
scraper = PlayStoreScraper(APP_IDS)
raw_reviews_df = scraper.fetch_all_reviews(target_per_app=TARGET_REVIEWS_PER_APP)


Starting scraping for: CBE
Collected 400 reviews so far (target for CBE: 400).

Starting scraping for: BOA
Collected 800 reviews so far (target for BOA: 400).

Starting scraping for: DASHEN
Collected 1200 reviews so far (target for DASHEN: 400).

Total reviews scraped: 1200


# Preprocess and save the scraped data

In [9]:
# Display some info about scraped data
if not raw_reviews_df.empty:
    print(f"\nShape of raw_reviews_df: {raw_reviews_df.shape}")
    print(raw_reviews_df.head())
    print("\nReview counts per bank (raw):")
    print(raw_reviews_df['bank'].value_counts())

    # 2. Preprocess Data
    preprocessor = ReviewPreprocessor(raw_reviews_df)
    cleaned_df = preprocessor.preprocess() # 

    print(f"\nShape of cleaned_df: {cleaned_df.shape}")
    print(cleaned_df.head())
    print("\nReview counts per bank (cleaned):")
    print(cleaned_df['bank'].value_counts())

    # Check for missing data percentage
    missing_percentage = (cleaned_df.isnull().sum().sum() / (len(cleaned_df) * len(cleaned_df.columns))) * 100
    print(f"\nOverall missing data percentage after cleaning: {missing_percentage:.2f}%") # Aim for <5% 

    # 3. Save to CSV 
    output_csv_path = '../data/cleaned_play_store_reviews.csv' # Make sure 'data' directory exists
    # Create data directory if it doesn't exist
    import os
    os.makedirs('../data', exist_ok=True)
    
    cleaned_df.to_csv(output_csv_path, index=False, encoding='utf-8')
    print(f"\nCleaned data saved to {output_csv_path}")
    
    # KPI Checks:
    print(f"\nKPI Check - Total reviews collected: {len(cleaned_df)}") # Should be 1200+ 
    for bank_name in APP_IDS.keys():
        count = len(cleaned_df[cleaned_df['bank'] == bank_name])
        print(f"Reviews for {bank_name}: {count}") # Should be 400+ per bank 


else:
    print("Scraping did not yield any data. Please check APP_IDS and network connectivity.")


Shape of raw_reviews_df: (1200, 5)
                                              review  rating  \
0                         So bad now and hard to use       5   
1  it is so amazing app. but, it is better to upd...       5   
2                                         v.good app       4   
3                                      very good app       1   
4           Very amazing app indeed. I'm enjoying it       5   

                 date bank             source  
0 2025-06-09 18:31:56  CBE  Google Play Store  
1 2025-06-09 16:20:06  CBE  Google Play Store  
2 2025-06-09 11:49:09  CBE  Google Play Store  
3 2025-06-09 01:24:23  CBE  Google Play Store  
4 2025-06-08 21:52:23  CBE  Google Play Store  

Review counts per bank (raw):
bank
CBE       400
BOA       400
DASHEN    400
Name: count, dtype: int64

--- Starting Preprocessing ---
Handling missing data...
Dropped 0 rows due to missing 'review' or 'rating'.
Removing duplicate reviews...
Dropped 0 duplicate rows.
Normalizing dates...
'